In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel , Field
from langchain_ollama import OllamaLLM  
model = OllamaLLM(model = 'llama3.2' , temperature=0.1  , base_url='http://shinigami:11111')

/home/harunachal/anaconda3/envs/llm/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
class robot_output(BaseModel):
    speak_to_human : bool = Field(description="set to true if there is a need for speaking with human" , default=True)
    output : str = Field(description="the sentence, the robot needs to speak to the human")
    mode : str  = Field(description="The navigation mode the robot needs to switch to  (back_off , move_forward)" , default="back_off")

In [3]:
import yaml
with open('ecohan/config/conversation.yaml' , 'r' ) as f :
    yaml_data = yaml.safe_load(f) 
 

In [5]:
yaml_data

{'conversation_robot_listener': {'task': 'you are a AI bot controlling navigation and speech strategies of a robot in a evironment with human moving. The humans you interact with might be in a hurry or relaxed. \nbased on human speech you need to understand their urgency, the robot has two navigation options 1) move forward (makes the robot continue its predefined task), 2) back off (blocks the robot from doing its task, allowing human to go first) \nyou must wisely choose one of these based on the level of urgency of the human. Pass the navigation strategy of \neither back off or move forward in the "mode" output field (required).optionally you can also decide to speak \nto the human about the situation. if you decide to speak, turn the "speak_to_human" output field to true, and fill in "speech" output field with the sentence that is socially acceptable to speak to a human.\nIf the human is passing urgent, you should back_off, and if they are relaxed or allow you to go, you should mov

In [6]:
parser = JsonOutputParser(pydantic_object=robot_output)
task= yaml_data['conversation_robot_listener']['task']
output_format = yaml_data['conversation_robot_listener']['output_format']

In [7]:
prompt = PromptTemplate.from_template(task )

In [8]:
prompt

PromptTemplate(input_variables=['human_speech', 'output_format'], input_types={}, partial_variables={}, template='you are a AI bot controlling navigation and speech strategies of a robot in a evironment with human moving. The humans you interact with might be in a hurry or relaxed. \nbased on human speech you need to understand their urgency, the robot has two navigation options 1) move forward (makes the robot continue its predefined task), 2) back off (blocks the robot from doing its task, allowing human to go first) \nyou must wisely choose one of these based on the level of urgency of the human. Pass the navigation strategy of \neither back off or move forward in the "mode" output field (required).optionally you can also decide to speak \nto the human about the situation. if you decide to speak, turn the "speak_to_human" output field to true, and fill in "speech" output field with the sentence that is socially acceptable to speak to a human.\nIf the human is passing urgent, you sho

In [9]:
output_format

'{\n  "speak_to_human" : "True if there is a need to speak to human",\n  "speech"  : "sentence to speak to the human",\n  "mode" : "type of navigation mode intended. (Required Field)"\n}\n'

In [10]:
chain = prompt |  model | parser


In [11]:
llm_response = chain.invoke({'human_speech'  : 'you can proceed' , 'output_format' : output_format })

In [12]:
llm_response

{'speak_to_human': False, 'speech': '', 'mode': 'move forward'}